<a href="https://colab.research.google.com/github/deovrat-n/EDA-groceries-dataset/blob/main/groceries_dataset_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# Groceries Dataset Exploratory Data Analysis (EDA)

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from pandas.api.types import CategoricalDtype

## Inspecting The Data

In [ ]:
groceries=pd.read_csv('../input/groceries-dataset/Groceries_dataset.csv')
groceries.head()

In [ ]:
groceries.info()

### Checking for Nulls

In [ ]:
groceries.isna().sum()

### General Description of Dataset

In [ ]:
groceries.describe(include='all')

### Changing Column Data Types

In [ ]:
groceries['Date']=pd.to_datetime(groceries['Date'])
groceries['Member_number']= groceries['Member_number'].astype('object')
groceries.info()

### Date Range of Dataset

In [ ]:
str(groceries['Date'].dt.date.min()) + '  to  ' +str(groceries['Date'].dt.date.max()) 

### Checking for Duplicates

In [ ]:
groceries.duplicated(keep='first').sum()

There are 759 potential duplicates identified. More investigation needs to be done.

In [ ]:
groceries[groceries.duplicated(keep=False)].sort_values(by='Member_number')

In [ ]:
groceries[groceries.duplicated(keep=False)].groupby(['Member_number','Date','itemDescription'],as_index=False).agg(DuplicateCount=('itemDescription','count')).sort_values(by='Member_number') 

**The tables above show potential duplicates. Though Member_number is a unique identifier, 
there is no data indicating that the potential duplicates are from the same invoice. 
Therefore,we proceed with the EDA.** 

### Adding Columns To Aid in EDA

In [ ]:
groceries['Weekdays'],groceries['Days'],groceries['Months'],groceries['Years']= groceries.Date.dt.strftime('%A'),groceries.Date.dt.strftime('%d'),groceries.Date.dt.strftime('%b'),groceries.Date.dt.strftime('%Y')
groceries.head()

### Number of Products in Dataset

In [ ]:
groceries['itemDescription'].nunique()

### Number of Customers

In [ ]:
groceries['Member_number'].nunique()

### Top 10 Products Sold

In [ ]:
# Using the value_counts function here. While for Top 10 Customers the groupby function was used.
# In this particular scenario both get the job done, but the value_counts function is simpler.

Top10Groceries=groceries.itemDescription.value_counts().sort_values(ascending=False)[0:10]

fig=px.bar(Top10Groceries,color=Top10Groceries.index,
           labels={'value':'Quantity Sold','index':'Grocery Items'})
fig.update_layout(showlegend=False,
                 title_text='Top 10 Groceries Sold',title_x=0.5,
                 title={'font': {'size':20}} )
fig.show()

### Top 10 Customer (Number of Items Bought)

In [ ]:
Top10Customers=groceries.groupby('Member_number').agg(PurchaseQuantity=('itemDescription','count')).sort_values(by='PurchaseQuantity',ascending=False)[0:10]

Top10Customers.plot(kind='bar',figsize=(15,7),legend=None)
plt.title('Top 10 Customers',fontsize=20)
plt.xlabel('Customer Number',fontsize=15)
plt.ylabel('Purchase Quantity',fontsize=15)
plt.show()

## Time Series Analysis

In [ ]:
groceries.set_index('Date',inplace=True)

### Daily Sales 

In [ ]:
groceries.resample('D')[['itemDescription']].count().plot(kind='line',figsize=(15,5),legend=None)
plt.title('Daily Sales- 2014 to 2015',fontsize=20)
plt.xlabel('Date',fontsize=14)
plt.ylabel('Daily Sales',fontsize=14)
plt.show()

### Monthly Sales- 2014 to 2015

In [ ]:
groceries.resample('M')[['itemDescription']].count().plot(kind='line',figsize=(15,7),legend=None )
plt.title('Monthly Sales- 2014 to 2015',fontsize=20)
plt.xlabel('Date',fontsize=14)
plt.ylabel('Monthly Sales',fontsize=14)
plt.show()

#### Lets Have a Different View of the above Monthly Purchases- 2014 to 2015 Graph, to better compare the two years.

In [ ]:
temp=groceries.groupby(['Years','Months'],as_index=False).agg(Sales=('itemDescription','count'))
temp.head()

In [ ]:
order_months=CategoricalDtype(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
temp['Months']=temp['Months'].astype(order_months)
temp=temp.sort_values(by='Months')
fig=px.line(temp,x='Months',y='Sales',color='Years')
fig.update_layout(title_text='Monthly Sales- 2014 to 2015',title_x=0.5,
                 title={'font': {'size':20}})

### Weekday Sales

In [ ]:
temp2=groceries.groupby(['Years','Weekdays'],as_index=False).agg(Sales=('itemDescription','count'))

order_days=CategoricalDtype(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
temp2['Weekdays']=temp2['Weekdays'].astype(order_days)
temp2=temp2.sort_values(by='Weekdays')

fig=px.line(temp2,x='Weekdays',y='Sales',color='Years')
fig.update_layout(title_text='Weekday Sales- 2014 to 2015',title_x=0.5,
                 title={'font': {'size':20}})

### Sales Per Days of the Month

In [ ]:
temp3=groceries.groupby(['Years','Days'],as_index=False).agg(Sales=('itemDescription','count'))

fig=px.line(temp3,x='Days',y='Sales',color='Years')
fig.update_layout(title_text='Sales Per Days of the Month',title_x=0.5,
                 title={'font':{'size':20}})
fig.show()

### Total Unique Customers Per Week

In [ ]:
temp4= groceries.groupby(['Years','Weekdays'],as_index=False).agg(DailyCustomers=('Member_number','nunique'))

order_days=CategoricalDtype(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
temp4['Weekdays']=temp4['Weekdays'].astype(order_days)
temp4=temp4.sort_values(by='Weekdays')

fig=px.line(temp4,x='Weekdays',y='DailyCustomers',color='Years')
fig.update_layout(title_text='Weekday Customers- 2014 & 2015',title_x=0.5,
                 title={'font': {'size':20}})

### Average Number of Unique Customers Per Weekday

In [ ]:
groceries=groceries.reset_index()

In [ ]:
temp5= groceries.groupby([pd.Grouper(key='Date',freq='D'),'Years','Months','Weekdays'],as_index=False).agg(UniqueCustomers=('Member_number','nunique'))
temp5

In [ ]:
temp6=temp5.groupby(['Years','Weekdays'],as_index=False)[['UniqueCustomers']].mean()

fig=px.bar(temp6,x='Weekdays',y='UniqueCustomers',color='Weekdays',facet_col='Years',
          category_orders={'Weekdays':['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']})
fig.update_layout(title_text='Avg Number of Unique Customers Per Weekday',title_x=0.5,
                 title={'font': {'size':20}},
                 showlegend=False)
fig.show()

**Across the board 2015 compared to 2014 has fewer average number of unique customers per weekday.**

### Customer Sales Per Weekday 

In [ ]:
temp6=groceries.groupby([pd.Grouper(key='Date',freq='D'),'Years','Weekdays'],as_index=False).agg(Sales=('itemDescription','count'),Customers=('Member_number','nunique'))
temp6

In [ ]:
temp7=temp6.groupby(['Years','Weekdays'],as_index=False).agg(Sales=('Sales','sum'),Customers=('Customers','sum')).eval('SalesPerCustomer=Sales/Customers').drop(columns=['Sales','Customers'])

fig=px.bar(temp7,x='Weekdays',y='SalesPerCustomer',color='Weekdays',facet_col='Years',
      category_orders={'Weekdays':['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']})

fig.update_layout(title_text='Weekday Sales Per Customer',title_x=0.5,
                 title={'font': {'size':20}},
                 showlegend=False)


fig.show()

**While 2014 had more average number of unique customers per weekday, 2015 has more sales per customer on a given weekday.**